# Food Insecurity Feature Engineering Notebook

This notebook replicates the cleaning and feature-engineering steps used to generate the engineered dataset from `FullData.csv`.  It removes duplicate county–year entries, aggregates numeric features, and creates additional variables such as demographic percentages, per-capita business counts, affordability ratios, log transforms, majority race and year-over-year changes.

In [1]:
from google.colab import files

uploaded = files.upload()  # This opens a file browser

Saving FullData.csv to FullData.csv


In [2]:

import pandas as pd
import numpy as np

# Load the supplied data
file_path = 'FullData.csv'
df = pd.read_csv(file_path)

print(f'Dataset loaded with shape: {df.shape}')
print(df.head())




Dataset loaded with shape: (49206, 30)
   FIPS         Rent  Year coc_number  Houseless_rate  Sheltered_rate  \
0  1073  1019.995960  2014     AL-500        0.001396        0.000961   
1  1117  1229.755051  2014     AL-500        0.001396        0.000961   
2  4003  1051.250000  2014     AZ-500        0.001469        0.000826   
3  4013  1095.670228  2014     AZ-502        0.001318        0.001084   
4  4019   928.546429  2014     AZ-501        0.002170        0.001705   

   Unsheltered_rate    State            County    TOT_POP  ...  \
0          0.000435  Alabama  Jefferson County   659972.0  ...   
1          0.000435  Alabama     Shelby County   206280.0  ...   
2          0.000643  Arizona    Cochise County   127314.0  ...   
3          0.000235  Arizona   Maricopa County  4093648.0  ...   
4          0.000465  Arizona       Pima County  1004229.0  ...   

               Low Threshold Type             High Threshold Type  \
0                            SNAP         Other Nutritio

In [3]:

# Determine numeric and categorical columns
numeric_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['FIPS','Year']]
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Define aggregation dictionary
# For numeric columns, take the mean; for categorical columns, pick the first non-null value
agg_dict = {col: 'mean' for col in numeric_cols}

def pick_first_nonnull(series):
    for val in series:
        if pd.notnull(val):
            return val
    return np.nan

for col in categorical_cols:
    agg_dict[col] = pick_first_nonnull

# Aggregate duplicates based on FIPS and Year
agg_df = df.groupby(['FIPS','Year'], as_index=False).agg(agg_dict)

# Ensure FIPS and Year are integers
agg_df['FIPS'] = agg_df['FIPS'].astype(int)
agg_df['Year'] = agg_df['Year'].astype(int)

print(f'Aggregated data shape: {agg_df.shape}')


Aggregated data shape: (48273, 30)


In [4]:

# Demographic percentages
agg_df['percent_male'] = agg_df['TOT_MALE'] / agg_df['TOT_POP']
agg_df['percent_female'] = agg_df['TOT_FEMALE'] / agg_df['TOT_POP']
agg_df['percent_white'] = agg_df['TOT_WHITE'] / agg_df['TOT_POP']
agg_df['percent_black'] = agg_df['TOT_BLACK'] / agg_df['TOT_POP']
agg_df['percent_native'] = agg_df['TOT_NATIVE'] / agg_df['TOT_POP']
agg_df['percent_asian'] = agg_df['TOT_ASIAN'] / agg_df['TOT_POP']
agg_df['percent_pacific'] = agg_df['TOT_PACIFIC'] / agg_df['TOT_POP']
agg_df['percent_latinx'] = agg_df['TOT_LATINX'] / agg_df['TOT_POP']
agg_df['percent_nonwhite'] = 1 - agg_df['percent_white']

# Gender ratio
agg_df['gender_ratio'] = agg_df['TOT_MALE'] / (agg_df['TOT_FEMALE'] + 1e-9)

# Houselessness composition
agg_df['prop_sheltered'] = agg_df['Sheltered_rate'] / (agg_df['Houseless_rate'] + 1e-9)
agg_df['prop_unsheltered'] = agg_df['Unsheltered_rate'] / (agg_df['Houseless_rate'] + 1e-9)

# Per capita food businesses
agg_df['wholesale_per_capita'] = agg_df['Num_wholesale'] / (agg_df['TOT_POP'] + 1e-9)
agg_df['restaurants_per_capita'] = agg_df['Num_restaraunts'] / (agg_df['TOT_POP'] + 1e-9)
agg_df['grocery_per_capita'] = agg_df['Num_grocery'] / (agg_df['TOT_POP'] + 1e-9)

# Business ratios
agg_df['grocery_to_restaurant_ratio'] = agg_df['Num_grocery'] / (agg_df['Num_restaraunts'] + 1e-9)
agg_df['wholesale_to_restaurant_ratio'] = agg_df['Num_wholesale'] / (agg_df['Num_restaraunts'] + 1e-9)
agg_df['grocery_to_wholesale_ratio'] = agg_df['Num_grocery'] / (agg_df['Num_wholesale'] + 1e-9)

# Employment features
agg_df['employment_rate'] = agg_df['Employed'] / (agg_df['Total_workforce'] + 1e-9)
agg_df['unemployment_rate_calc'] = agg_df['Unemployed'] / (agg_df['Total_workforce'] + 1e-9)
agg_df['workforce_participation_rate'] = agg_df['Total_workforce'] / (agg_df['TOT_POP'] + 1e-9)

# Food affordability
agg_df['monthly_food_cost'] = agg_df['Cost Per Meal'] * 3 * 30
agg_df['food_to_rent_ratio'] = agg_df['monthly_food_cost'] / (agg_df['Rent'] + 1e-9)

# Log transforms
agg_df['log_rent'] = np.log1p(agg_df['Rent'])
agg_df['log_cost_per_meal'] = np.log1p(agg_df['Cost Per Meal'])
agg_df['log_total_workforce'] = np.log1p(agg_df['Total_workforce'])

# Majority race determination

def determine_majority_race(row):
    race_shares = {
        'white': row['percent_white'],
        'black': row['percent_black'],
        'latinx': row['percent_latinx'],
        'native': row['percent_native'],
        'asian': row['percent_asian'],
        'pacific': row['percent_pacific']
    }
    return max(race_shares, key=lambda k: race_shares[k] if pd.notnull(race_shares[k]) else -1)

agg_df['majority_race'] = agg_df.apply(determine_majority_race, axis=1)

# Year-over-year differences
agg_df = agg_df.sort_values(['FIPS', 'Year'])
agg_df['delta_FI_rate'] = agg_df.groupby('FIPS')['FI Rate'].diff()
agg_df['delta_unemployment_rate'] = agg_df.groupby('FIPS')['Unemployment_rate'].diff()
agg_df['delta_rent'] = agg_df.groupby('FIPS')['Rent'].diff()
agg_df['delta_cost_per_meal'] = agg_df.groupby('FIPS')['Cost Per Meal'].diff()

print('Engineered features added. Data now has', agg_df.shape[1], 'columns.')


Engineered features added. Data now has 61 columns.


In [ ]:

# Save the engineered dataset to CSV
output_path = 'engineered_data_from_notebook.csv'
agg_df.to_csv(output_path, index=False)

print('Engineered dataset saved to', output_path)
